**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
suppressMessages(suppressWarnings(library("DESeq2")))
suppressMessages(suppressWarnings(library("edgeR")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



**Set global variable**

In [2]:
txt_fdiry = file.path(FD_RES, "assay_fcc")
vec = dir(txt_fdiry)
for (txt in vec) {cat(txt, "\n")}

CRISPRi_FlowFISH_K562_Riley_JinWoo 
CRISPRi_Growth_K562_Gersbach_JinWoo 
MPRA_Lenti_K562_Nadav_Vikram 
MPRA_Tiling_K562_Tewhey_Hannah 
STARR_ATAC_K562_Reddy_KS274 
STARR_ATAC_K562_Reddy_KS91 
STARR_ATAC_K562_Reddy_KSMerge 
STARR_WHG_K562_Reddy_A001 


In [3]:
VEC_TXT_ASSAY = c(
    "STARR_ATAC_K562_Reddy_KS91",
    "STARR_ATAC_K562_Reddy_KS274",
    "STARR_ATAC_K562_Reddy_KSMerge",
    "STARR_WHG_K562_Reddy_A001"
)

**Helper function**

In [4]:
get_fpkm = function(mat, vec_num_length){
    ### normlaized by size
    fun = function(x){x * 10^6 / sum(x)}
    mat = apply(mat, 2, fun)
    
    ### normlaized by length
    vec = vec_num_length
    fun = function(x){x/vec}
    mat = apply(mat, 2, fun)

    return(mat)
}

get_tpm  = function(mat, vec_num_length){
    ### normlaized by length
    vec = vec_num_length
    fun = function(x){x/vec}
    mat = apply(mat, 2, fun)
    
    ### normlaized by size
    fun = function(x){x * 10^6 / sum(x)}
    mat = apply(mat, 2, fun)
    
    return(mat)
}

## Execute

In [9]:
txt_fpath = "/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/astarr_macs_input_union/STARR_ATAC_K562_Reddy_KS91/overlap_count/summary/data.count_column.raw.WGS.rds"
lst_dat = readRDS(txt_fpath)

Warning message in gzfile(file, "rb"):
“cannot open compressed file '/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_coverage_fcc/astarr_macs_input_union/STARR_ATAC_K562_Reddy_KS91/overlap_count/summary/data.count_column.raw.WGS.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [11]:
###
vec_txt_assay  = VEC_TXT_ASSAY
vec_txt_region = c("fcc_astarr_macs_input_overlap", "fcc_astarr_macs_input_union")

###
for (txt_assay in vec_txt_assay){
    for (txt_region_label in vec_txt_region) {
        
        ### show progress
        cat("Assay        ", txt_assay,         "\n")
        cat("Region Label ", txt_region_label,  "\n")
        cat("\n")
        flush.console()

        ### get column and column data
        txt_fdiry = file.path(
            FD_RES, 
            "region_coverage_fcc",
            txt_region_label,
            txt_assay, 
            "overlap_count", 
            "summary")

        ###
        txt_fname = "data.count_column.raw.WGS.rds"
        txt_fpath = file.path(txt_fdiry, txt_fname)

        lst_dat = readRDS(txt_fpath)
        dat_cnt = lst_dat$data_cnt
        dat_col = lst_dat$data_col
        dat_reg = dat_cnt %>% dplyr::select(Chrom, ChromStart, ChromEnd, Region)
        
        ###
        txt_fname = "data.deseq2.rds"
        txt_fpath = file.path(txt_fdiry, txt_fname)

        dds = readRDS(txt_fpath)
        
        ###
        txt_fname = "data.edger.rds"
        txt_fpath = file.path(txt_fdiry, txt_fname)

        lst_dge = readRDS(txt_fpath)

        ### ========================================
        ### Calculate: Screened
        ### ----------------------------------------
        
        lst = lst_dge
        lst = lapply(lst, function(dge){
            idx = filterByExpr(dge)
            dat = data.frame(
                Region = names(idx),
                Screen = idx
            )
            return(dat)
        })
        
        dat = bind_rows(lst, .id = "Set")
        dat = dat %>% tidyr::spread(Set, Screen)
        dat = dplyr::left_join(dat_reg, dat, by = "Region")
        
        dat_screen = dat
        
        ### ========================================
        ### Calculate DESeq2 Counts & Log2FC
        ### ----------------------------------------

        ### DESeq2 normalized counts
        mat = counts(dds, normalized=TRUE)
        dat = mat %>% as.data.frame %>% rownames_to_column("Region")
        dat = dplyr::left_join(dat_reg, dat, by = "Region")

        mat_dds_count = mat
        dat_dds_count = dat

        ### Deseq2 results
        res = results(dds, contrast = c("Group", "Output", "Input"))
        dat = res %>% as.data.frame %>% rownames_to_column("Region")
        dat = dplyr::left_join(dat_reg, dat, by = "Region")
        
        dat_dds_result = dat

        ### ========================================
        ### Calculate CPM
        ### ----------------------------------------
        
        ### ========================================
        ### Calculate TPM & FPKM
        ### ----------------------------------------

        ### init and get region length
        dat = dat_cnt
        dat = dat %>% dplyr::mutate(Length = ChromEnd - ChromStart)

        ### arrange count into matrix
        mat = dat %>% 
           dplyr::select(
               Region, 
               starts_with("Input"), 
               starts_with("Output")) %>% 
           column_to_rownames(var = "Region")
        
        ### calculate TPM and FPKM
        mat_tpm  = get_tpm(mat, dat$Length)
        mat_fpkm = get_fpkm(mat, dat$Length)

        ### arrange matrix into dataframe
        mat = mat_tpm
        dat = mat %>% as.data.frame %>% rownames_to_column("Region")
        dat = dplyr::left_join(dat_reg, dat, by = "Region")
        dat_tpm = dat

        mat = mat_fpkm
        dat = mat %>% as.data.frame %>% rownames_to_column("Region")
        dat = dplyr::left_join(dat_reg, dat, by = "Region")
        dat_fpkm = dat
        
        ### ========================================
        ### Calculate mean TPM & FPKM
        ### ----------------------------------------
        
        ### convert to tidy data
        lst = list(
            "TPM"   = mat_tpm,
            "FPKM"  = mat_fpkm,
            "DESeq" = mat_dds_count
        )
        lst = lapply(lst, function(mat){
            dat = mat %>% 
                as.data.frame %>% 
                rownames_to_column(var = "Region") %>% 
                tidyr::gather(Sample, Value, -Region)
            return(dat)
        })
        dat = bind_rows(lst, .id = "Method")
        
        ### calculate mean by group (Input & Output)
        tmp = dat_col %>% dplyr::select(Sample, Group)
        dat = dplyr::left_join(dat, tmp, by="Sample")
        dat = dat %>%
            dplyr::group_by(Region, Method, Group) %>%
            dplyr::summarize(Mean = mean(Value), .groups = "drop") %>%
            dplyr::mutate(Name = paste(Group, Method, sep = "_")) %>%
            dplyr::select(Region, Name, Mean) %>%
            tidyr::spread(Name, Mean)
        
        dat = dplyr::left_join(dat_reg, dat, by = "Region")
        dat_summary = dat

        ### ========================================
        ### Save results
        ### ----------------------------------------
        
        txt_fname = "matrix.count.TPM.WGS.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat = dat_tpm
        write_tsv(dat, txt_fpath)
        
        txt_fname = "matrix.count.FPKM.WGS.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat = dat_fpkm
        write_tsv(dat, txt_fpath)
        
        txt_fname = "matrix.count.deseq.WGS.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat = dat_dds_count
        write_tsv(dat, txt_fpath)
        
        txt_fname = "result.coverage.Log2FC.deseq.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat = dat_dds_result
        write_tsv(dat, txt_fpath)
        
        txt_fname = "result.coverage.summary.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat = dat_summary
        write_tsv(dat, txt_fpath)
        
        txt_fname = "result.coverage.screened.tsv"
        txt_fpath = file.path(txt_fdiry, txt_fname)
        dat = dat_screen
        write_tsv(dat, txt_fpath)
        
    }
}

Assay         STARR_ATAC_K562_Reddy_KS91 
Region Label  fcc_astarr_macs_input_overlap 



Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”
Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”


Assay         STARR_ATAC_K562_Reddy_KS91 
Region Label  fcc_astarr_macs_input_union 



Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”
Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”


Assay         STARR_ATAC_K562_Reddy_KS274 
Region Label  fcc_astarr_macs_input_overlap 



...

Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”
Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”


Assay         STARR_WHG_K562_Reddy_A001 
Region Label  fcc_astarr_macs_input_overlap 



Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”
Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”


Assay         STARR_WHG_K562_Reddy_A001 
Region Label  fcc_astarr_macs_input_union 



Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”
Warning message in filterByExpr.DGEList(dge):
“All samples appear to belong to the same group.”


## Review

In [56]:
head(dat_dds_count, 3)

Chrom,ChromStart,ChromEnd,Region,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,chr1:10038-10405,85.29216,94.85917,103.92258,104.02099,82.94011,106.93828,26.91726,34.93514,11.10767,15.78996
chr1,14282,14614,chr1:14282-14614,62.35151,63.69770,65.16457,62.23090,57.31255,66.44709,43.06761,66.95901,72.19984,64.73885
chr1,16025,16338,chr1:16025-16338,72.93950,86.61054,74.53464,77.22082,82.94011,85.13533,43.06761,20.37883,41.65375,28.42194


In [57]:
head(dat_dds_result, 3)

Chrom,ChromStart,ChromEnd,Region,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,chr1:10038-10405,66.67233,-2.23485278,0.2790669,-8.0083052,1.163000e-15,8.299026e-15
chr1,14282,14614,chr1:14282-14614,62.41696,0.03179054,0.1644506,0.1933136,8.467134e-01,8.816469e-01
chr1,16025,16338,chr1:16025-16338,61.29031,-1.33934074,0.2226910,-6.0143468,1.806134e-09,8.654528e-09


In [59]:
head(dat_tpm, 3)

Chrom,ChromStart,ChromEnd,Region,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,chr1:10038-10405,3.445338,3.888148,4.290341,4.273202,3.397390,4.345809,0.8436118,1.171350,0.3693823,0.4884531
chr1,14282,14614,chr1:14282-14614,2.784182,2.886126,2.973866,2.825963,2.595126,2.984980,1.4920749,2.481768,2.6541010,2.2137812
chr1,16025,16338,chr1:16025-16338,3.454675,4.162517,3.607961,3.719535,3.983521,4.056664,1.5826481,0.801171,1.6241611,1.0309013


In [60]:
head(dat_fpkm, 3)

Chrom,ChromStart,ChromEnd,Region,Input.rep1,Input.rep2,Input.rep3,Input.rep4,Input.rep5,Input.rep6,Output.rep1,Output.rep2,Output.rep3,Output.rep4
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,chr1:10038-10405,0.003629544,0.004114802,0.004548045,0.004522405,0.003592098,0.004579404,0.0008557153,0.0012171316,0.0003817082,0.000488374
chr1,14282,14614,chr1:14282-14614,0.002933039,0.003054368,0.003152495,0.002990766,0.002743856,0.003145428,0.0015134820,0.0025787671,0.0027426652,0.002213423
chr1,16025,16338,chr1:16025-16338,0.003639380,0.004405164,0.003824677,0.003936449,0.004211821,0.004274716,0.0016053547,0.0008324843,0.0016783574,0.001030734


In [58]:
head(dat_summary, 3)

Chrom,ChromStart,ChromEnd,Region,Input_FPKM,Input_TPM,Output_FPKM,Output_TPM
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,chr1:10038-10405,0.004164383,3.940038,0.0007357323,0.7181993
chr1,14282,14614,chr1:14282-14614,0.003003325,2.841707,0.0022620843,2.2104314
chr1,16025,16338,chr1:16025-16338,0.004048701,3.830812,0.0012867327,1.2597204
